In [1]:
import pandas as pd
from datetime import datetime

pd.options.display.max_rows=500

In [2]:
import os

In [3]:
from pycountry import countries

In [4]:
pwd

'/Users/amlvt225/Code/GitHub/coronavirus-analysis/coronavirus/jupyter_notebooks'

In [5]:
cd ../python_scripts/

/Users/amlvt225/Code/GitHub/coronavirus-analysis/coronavirus/python_scripts


In [6]:
from hopkins_cleaner import HopkinsDataFull

In [7]:
self = HopkinsDataFull()

In [8]:
self.run()

/Users/amlvt225/Code/GitHub/coronavirus-analysis/coronavirus/python_scripts/us_data_cleaner.py:113: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  self.data = pd.concat([df1, df2], axis=0)
/Users/amlvt225/Code/GitHub/coronavirus-analysis/coronavirus/python_scripts/hopkins_cleaner.py:252: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ].rank(ascending=True)


In [9]:
df = self.data

In [10]:
df.loc[df['province_or_state'] == 'VA'].tail(10)

,country_or_region,date,province_or_state,running_total_cases,state_and_country,running_total_cases_prev_day,daily_new_cases,running_total_recoveries,running_total_recoveries_prev_day,daily_new_recoveries,...,state_name,latitude,longitude,first_case_state_rank,first_case_country_rank,country_code_2,country_code_3,country_population_2018,country_median_age,country_running_agg
13454,US,2020-03-06,VA,0,VA-US,0.0,0.0,0,0.0,0.0,...,Virginia,37.926868,-78.024902,NaN,33.0,US,USA,327167434.0,38.1,262
13455,US,2020-03-07,VA,0,VA-US,0.0,0.0,0,0.0,0.0,...,Virginia,37.926868,-78.024902,NaN,34.0,US,USA,327167434.0,38.1,402
13456,US,2020-03-08,VA,2,VA-US,0.0,2.0,0,0.0,0.0,...,Virginia,37.926868,-78.024902,NaN,35.0,US,USA,327167434.0,38.1,518
13457,US,2020-03-09,VA,2,VA-US,2.0,0.0,0,0.0,0.0,...,Virginia,37.926868,-78.024902,NaN,36.0,US,USA,327167434.0,38.1,583
14004,US,2020-03-10,VA,7,VA-US,2.0,5.0,0,0.0,0.0,...,Virginia,37.926868,-78.024902,1.0,37.0,US,USA,327167434.0,38.1,959
14005,US,2020-03-11,VA,9,VA-US,7.0,2.0,0,0.0,0.0,...,Virginia,37.926868,-78.024902,2.0,38.0,US,USA,327167434.0,38.1,1281
14006,US,2020-03-12,VA,17,VA-US,9.0,8.0,0,0.0,0.0,...,Virginia,37.926868,-78.024902,3.0,39.0,US,USA,327167434.0,38.1,1663
14007,US,2020-03-13,VA,30,VA-US,17.0,13.0,0,0.0,0.0,...,Virginia,37.926868,-78.024902,4.0,40.0,US,USA,327167434.0,38.1,2179
14008,US,2020-03-14,VA,41,VA-US,30.0,11.0,0,0.0,0.0,...,Virginia,37.926868,-78.024902,5.0,41.0,US,USA,327167434.0,38.1,2727
14009,US,2020-03-15,VA,45,VA-US,41.0,4.0,0,0.0,0.0,...,Virginia,37.926868,-78.024902,6.0,42.0,US,USA,327167434.0,38.1,3499


In [11]:
df.loc[df['province_or_state'] == 'VA']

,country_or_region,date,province_or_state,running_total_cases,state_and_country,running_total_cases_prev_day,daily_new_cases,running_total_recoveries,running_total_recoveries_prev_day,daily_new_recoveries,...,state_name,latitude,longitude,first_case_state_rank,first_case_country_rank,country_code_2,country_code_3,country_population_2018,country_median_age,country_running_agg
13410,US,2020-01-22,VA,0,VA-US,0.0,0.0,0,0.0,0.0,...,Virginia,37.926868,-78.024902,NaN,NaN,US,USA,327167434.0,38.1,1
13411,US,2020-01-23,VA,0,VA-US,0.0,0.0,0,0.0,0.0,...,Virginia,37.926868,-78.024902,NaN,NaN,US,USA,327167434.0,38.1,1
13412,US,2020-01-24,VA,0,VA-US,0.0,0.0,0,0.0,0.0,...,Virginia,37.926868,-78.024902,NaN,NaN,US,USA,327167434.0,38.1,2
13413,US,2020-01-25,VA,0,VA-US,0.0,0.0,0,0.0,0.0,...,Virginia,37.926868,-78.024902,NaN,NaN,US,USA,327167434.0,38.1,2
13414,US,2020-01-26,VA,0,VA-US,0.0,0.0,0,0.0,0.0,...,Virginia,37.926868,-78.024902,NaN,NaN,US,USA,327167434.0,38.1,5
13415,US,2020-01-27,VA,0,VA-US,0.0,0.0,0,0.0,0.0,...,Virginia,37.926868,-78.024902,NaN,NaN,US,USA,327167434.0,38.1,5
13416,US,2020-01-28,VA,0,VA-US,0.0,0.0,0,0.0,0.0,...,Virginia,37.926868,-78.024902,NaN,NaN,US,USA,327167434.0,38.1,5
13417,US,2020-01-29,VA,0,VA-US,0.0,0.0,0,0.0,0.0,...,Virginia,37.926868,-78.024902,NaN,NaN,US,USA,327167434.0,38.1,5
13418,US,2020-01-30,VA,0,VA-US,0.0,0.0,0,0.0,0.0,...,Virginia,37.926868,-78.024902,NaN,NaN,US,USA,327167434.0,38.1,5
13419,US,2020-01-31,VA,0,VA-US,0.0,0.0,0,0.0,0.0,...,Virginia,37.926868,-78.024902,NaN,NaN,US,USA,327167434.0,38.1,7


In [18]:
yesterday.sort_values('running_total_cases', ascending=False)[['country_or_region','running_total_cases','running_total_recoveries','running_total_deaths','country_population_2018','country_median_age']]

,country_or_region,running_total_cases,running_total_recoveries,running_total_deaths,country_population_2018,country_median_age
5615,China,67794,54288,3085,1.392730e+09,37.4
917,Italy,24747,2335,1441,6.043128e+07,45.5
5669,Iran,13938,4590,724,8.180027e+07,30.3
5723,"Korea, South",8162,510,75,NaN,NaN
1025,Spain,7798,517,289,4.672375e+07,42.7
647,Germany,5795,46,11,8.292792e+07,47.1
5777,France,4499,12,91,6.698724e+07,41.4
1727,Switzerland,2200,4,14,8.516543e+06,42.4
5831,China,1360,1304,8,1.392730e+09,37.4
5885,China,1273,1250,22,1.392730e+09,37.4


In [12]:
df.to_csv('../output_data/HOPKINS_CLEANED.csv', index=False)

# QA 

#### Make sure all US states have the same number of days 

In [ ]:
us = df.loc[df['country_or_region'] == 'US']

In [ ]:
us.groupby(['province_or_state'])['daily_new_cases'].size().reset_index()

#### Total US cases 

In [ ]:
us['daily_new_cases'].sum()

In [ ]:
df.head(2)

In [13]:
max_date = df['date'].max()

In [14]:
yesterday = df.loc[df['date'] == max_date]
yesterday['running_total_cases'].sum(), yesterday['running_total_recoveries'].sum(), yesterday['running_total_deaths'].sum()

(167449, 76034, 6066)

In [ ]:
df['daily_new_cases'].sum(), df['daily_new_recoveries'].sum(), df['daily_new_deaths'].sum()

In [ ]:
df.loc[df['province_or_state'] == 'WA'].tail(5)

In [ ]:
df.loc[df['province_or_state'] == 'VA'].tail(5)

In [ ]:
df.loc[df['province_or_state'] == 'CA'].tail(5)

## QA of Data 

In [ ]:
# This indicates a shift in the data
df.loc[df['daily_diff_cases'] < 0]['date'].value_counts()

In [ ]:
# This indicates a shift in the data
df_final.loc[df_final['daily_diff_cases'] < 0]['province_state'].value_counts()

In [ ]:
df_final.loc[df_final['daily_diff_cases'] < 0]

In [ ]:
df_final.loc[(df_final['daily_diff_cases'] < 0) &
      (df_final['province_state'] == 'CA')]

In [ ]:
df_final.loc[df_final['country_region'] == 'China'].head(200).sort_values('rank')

In [ ]:
df_final.country_region.value_counts()